# Basic RAG

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [4]:
# 1. Load documents
print("Loading documents...")
loader = DirectoryLoader("data/scientists_bios")
docs = loader.load()
print(f"Loaded {len(docs)} documents")

Loading documents...
Loaded 5 documents


In [5]:
docs

[Document(metadata={'source': 'data/scientists_bios/Ada Lovelace.txt'}, page_content='# Ada Lovelace (1815\n\n1852)\n\nAugusta Ada King, Countess of Lovelace (née Byron), was an English mathematician and writer, chiefly known for her work on Charles Babbage\'s proposed mechanical general-purpose computer, the Analytical Engine. She is often regarded as the world\'s first computer programmer, having published the first algorithm intended to be processed by a machine.\n\n## Early Life and Education\n\nAda Lovelace was born on December 10, 1815, in London, England. She was the only legitimate child of the poet Lord Byron and his wife Anne Isabella Milbanke, a mathematically inclined heiress and philanthropist. Ada\'s parents separated when she was just one month old, and she never met her father, who died in Greece when she was eight years old.\n\nAda\'s mother, determined that her daughter would not develop the "insanity" or poetic temperament of Byron, ensured Ada received a rigorous ed

In [10]:
# 2. Create embeddings and vector store
embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-small")
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents=docs)

['a5caca2c-96a7-403f-96e4-c3281b2d6c49',
 '3b975b7d-2ba8-4854-af46-13f41bbefca4',
 '60a2e532-e99c-4d21-bed9-bc07429accef',
 '00061b56-37cc-4191-a321-00074dc4f865',
 '838b370a-c84a-40ef-a951-8bb0402dbc66']

In [13]:
vector_store.__dict__.keys()

dict_keys(['store', 'embedding'])

In [22]:
vector_store.store[list(vector_store.store.keys())[0]].keys()

dict_keys(['id', 'vector', 'text', 'metadata'])

In [23]:
# 3. Create retriever
retriever = vector_store.as_retriever()

In [24]:
# 4. Create LLM
llm = AzureChatOpenAI(model="gpt-5-nano")

In [25]:
# 5. Create prompt template
prompt = ChatPromptTemplate.from_template("""
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:
""")

In [26]:
# 6. Create RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
# Answer a single question using the RAG system
question = "What was Ada Lovelace's contribution to computer programming?"
response = rag_chain.invoke(question)
print(f"Q: {question}")
print(f"A: {response}")

Q: What was Ada Lovelace's contribution to computer programming?
A: Ada Lovelace is often regarded as the world's first computer programmer. She published the first algorithm intended to be processed by a machine (the Analytical Engine) to compute Bernoulli numbers. Her notes also anticipated future computing possibilities beyond calculation, such as music and graphics.


In [28]:
# Sample questions for testing
questions = [
    "What was Ada Lovelace's contribution to computer programming?",
    "How did Einstein develop his theory of relativity?",
    "What was Newton's most important discovery?"
]

# Answer multiple questions using the RAG system

for i, question in enumerate(questions, 1):
    print(f"\nQ{i}: {question}")
    print("-" * 40)
    response = rag_chain.invoke(question)
    print(f"A{i}: {response}")


Q1: What was Ada Lovelace's contribution to computer programming?
----------------------------------------
A1: Ada Lovelace is regarded as the world’s first computer programmer. She published an algorithm for Charles Babbage’s Analytical Engine to compute Bernoulli numbers, essentially the first published computer program. Her notes also foresaw broader computing possibilities beyond pure calculation, such as music and graphics.

Q2: How did Einstein develop his theory of relativity?
----------------------------------------
A2: - He published the special theory of relativity in 1905. 
- He then expanded it to the general theory of relativity in 1915. 
- These theories revolutionized our understanding of space, time, gravity, and the universe.

Q3: What was Newton's most important discovery?
----------------------------------------
A3: Newton’s most important discovery is the law of universal gravitation. It states that all bodies attract each other with a force proportional to the pro

Jaki mamy tutaj problem? Mamy stworzony embedding dla **całego dokumentu**. W promptcie, jako `{context}` podawana jest cała zawartość pliku, co generuje dużo tokenów wejściowych.

Rozwiązanie: Trzymanie w wektorowej bazie danych tzw. chunków, a nie całych dokumentów.

Zadanie: Samodzielne uruchomienie i analiza skryptu `2. minimal_rag_wtih_chunking.py`.